In [3]:
# Install libraries
! pip install transformers evaluate accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.7 MB/s eta 0:00:00


## Load Data

In [5]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="/content/train.csv", split="train")
dataset = dataset.train_test_split(test_size=0.1)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-e79e93742807aefd/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


In [3]:
dataset["test"][0]

{'rowid': 7081,
 'content': "i can fuck with birdwalk..cuz it's a dance beat..but that stupid yes sir..HELL NO!! eat a d*. LOL gucci bandana is aite. not hott",
 'label': 0}

# Train

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")

In [7]:
def preprocess_function(examples):
...     return tokenizer(examples["content"], truncation=True)
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/9882 [00:00<?, ? examples/s]

Map:   0%|          | 0/1098 [00:00<?, ? examples/s]

In [8]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

In [10]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-cased", num_labels=2)

training_args = TrainingArguments(
    output_dir="distrollbert-cased",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.2,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.433800,0.386964,0.818761
2,0.363800,0.385001,0.826047


Epoch,Training Loss,Validation Loss,Accuracy
1,0.433800,0.386964,0.818761
2,0.363800,0.385001,0.826047
3,0.307700,0.489517,0.797814
4,0.241200,0.643582,0.801457


TrainOutput(global_step=2472, training_loss=0.30548907406507575, metrics={'train_runtime': 10707.3533, 'train_samples_per_second': 3.692, 'train_steps_per_second': 0.231, 'total_flos': 506601903030384.0, 'train_loss': 0.30548907406507575, 'epoch': 4.0})

In [9]:
trainer.evaluate()

{'eval_loss': 0.38500121235847473,
 'eval_accuracy': 0.8260473588342441,
 'eval_runtime': 91.194,
 'eval_samples_per_second': 12.04,
 'eval_steps_per_second': 0.757,
 'epoch': 4.0}

## Hyperparameter Search

In [10]:
! pip install optuna
! pip install ray[tune]

In [11]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained("distilbert-base-cased", num_labels=2)

trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=tokenized_dataset["train"].shard(index=1, num_shards=10),
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32]),
        "weight_decay": trial.suggest_float("weight_decay", 0.01, 0.2, step=0.02)
    }

best_run = trainer.hyperparameter_search(n_trials=10, backend="optuna", hp_space=optuna_hp_space, direction="maximize")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[I 2023-07-20 19:02:12,270] A new study created in memory with name: no-name-03cf2d3f-9139-450a-b322-ef207dc41b60
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:685: UserWarning: The distribution is specified by [0.01, 0.2] and step=0.02, but the range is not divisible by `step`. It will be replaced by [0.01, 0.19].
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream tas

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.471214,0.820583
2,No log,0.456305,0.820583
3,No log,0.444290,0.820583
4,No log,0.440298,0.820583


[I 2023-07-20 19:33:18,977] Trial 0 finished with value: 0.8205828779599271 and parameters: {'learning_rate': 1.1306587842221592e-05, 'per_device_train_batch_size': 32, 'weight_decay': 0.05}. Best is trial 0 with value: 0.8205828779599271.
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:685: UserWarning: The distribution is specified by [0.01, 0.2] and step=0.02, but the range is not divisible by `step`. It will be replaced by [0.01, 0.19].
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.657217,0.820583
2,No log,0.609603,0.820583
3,No log,0.582799,0.820583
4,No log,0.573477,0.820583


[I 2023-07-20 20:04:19,845] Trial 1 finished with value: 0.8205828779599271 and parameters: {'learning_rate': 1.6539533347858152e-06, 'per_device_train_batch_size': 32, 'weight_decay': 0.13}. Best is trial 0 with value: 0.8205828779599271.
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:685: UserWarning: The distribution is specified by [0.01, 0.2] and step=0.02, but the range is not divisible by `step`. It will be replaced by [0.01, 0.19].
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version

...

[I 2023-07-20 23:01:42,089] Trial 7 finished with value: 0.8205828779599271 and parameters: {'learning_rate': 1.4890971426290461e-06, 'per_device_train_batch_size': 32, 'weight_decay': 0.03}. Best is trial 0 with value: 0.8205828779599271.
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:685: UserWarning: The distribution is specified by [0.01, 0.2] and step=0.02, but the range is not divisible by `step`. It will be replaced by [0.01, 0.19].
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.417795,0.816940


[I 2023-07-20 23:09:14,670] Trial 8 pruned. 
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:685: UserWarning: The distribution is specified by [0.01, 0.2] and step=0.02, but the range is not divisible by `step`. It will be replaced by [0.01, 0.19].
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.524761,0.820583
2,No log,0.474371,0.820583
3,No log,0.466824,0.820583
4,No log,0.465155,0.820583


[I 2023-07-20 23:42:21,794] Trial 9 finished with value: 0.8205828779599271 and parameters: {'learning_rate': 5.87925508805654e-06, 'per_device_train_batch_size': 32, 'weight_decay': 0.05}. Best is trial 0 with value: 0.8205828779599271.


### Retrain with Best Parameters

In [9]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-cased", num_labels=2)

training_args = TrainingArguments(
    output_dir="distrollbert-cased",
    learning_rate=1.5e-05,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.05,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.425475,0.806011
2,0.410400,0.411065,0.807832
3,0.410400,0.428844,0.795082
4,0.324100,0.505133,0.812386
5,0.248000,0.538062,0.789617


TrainOutput(global_step=1545, training_loss=0.32498434307505786, metrics={'train_runtime': 16317.2581, 'train_samples_per_second': 3.028, 'train_steps_per_second': 0.095, 'total_flos': 725947217173680.0, 'train_loss': 0.32498434307505786, 'epoch': 5.0})

In [11]:
trainer.evaluate()

{'eval_loss': 0.41106489300727844,
 'eval_accuracy': 0.807832422586521,
 'eval_runtime': 152.3208,
 'eval_samples_per_second': 7.208,
 'eval_steps_per_second': 0.23,
 'epoch': 5.0}

## Inference

In [18]:
def get_prediction(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, truncation=True, return_tensors="pt")
    # perform inference with our model
    outputs = model(**inputs)
    # get output probabilities with softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    return probs.argmax()

In [16]:
import pandas as pd

reddit_df = pd.read_csv('reddit_trolls.csv')
test_df = pd.read_csv('test.csv')
test_df

,rowid,content
0,1,'Night Guy' is ALL motivated @ 11pm but he kno...
1,2,- :) And yeah I know I think I set off ever...
2,3,- and btw I hate the new Pepsi logo/can. Maybe...
3,4,- Austin Girl's Fat Bastard (Beagle) fibs on r...
4,5,- can be quite a rough routine but I'm gonna r...
...,...,...
3655,3656,Your son is too damn cute...and I don't even l...
3656,3657,yup! Fat Albert is next
3657,3658,Yup! Wouldn't want me doing your wedding pics....
3658,3659,Yup. On Thanksgiving day. Fucking cops. ;)


In [ ]:
# Generalization to Reddit Data
pred = []
for index, row in reddit_df.iterrows():
  pred.append(get_prediction(row["content"]).item())

reddit_df["distrollbert_pred"] = pred
reddit_df.to_csv("reddit_distrollbert_predictions2.csv")

In [19]:
# Inference on Twitter test set
pred = []
for index, row in test_df.iterrows():
  pred.append(get_prediction(row["content"]).item())

test_df["label"] = pred
test_df

,rowid,content,label
0,1,'Night Guy' is ALL motivated @ 11pm but he kno...,0
1,2,- :) And yeah I know I think I set off ever...,0
2,3,- and btw I hate the new Pepsi logo/can. Maybe...,0
3,4,- Austin Girl's Fat Bastard (Beagle) fibs on r...,0
4,5,- can be quite a rough routine but I'm gonna r...,0
...,...,...,...
3655,3656,Your son is too damn cute...and I don't even l...,0
3656,3657,yup! Fat Albert is next,0
3657,3658,Yup! Wouldn't want me doing your wedding pics....,0
3658,3659,Yup. On Thanksgiving day. Fucking cops. ;),1


In [20]:
test_df.to_csv("distrollbert_mod_predictions3.csv")

In [13]:
# If you want to load the trained model from the Huggingface hub:

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("rimonim/distrollbert-cased")
model = AutoModelForSequenceClassification.from_pretrained("rimonim/distrollbert-cased")